In [ ]:
import itertools
from functools import lru_cache
import math
from collections import defaultdict
from fractions import Fraction

In [ ]:
@lru_cache(maxsize=None)
def A(i, n=3, c=3):
  """Probability of being absent at least 3 consecutive days."""
  if i < n:
    return 0
  elif i == n:
    return Fraction(1, c**i)
  return A(i-1, n, c) + (1 - A(i-n-1, n, c))*Fraction(c-1, c)*Fraction(1, c**n)

@lru_cache(maxsize=None)
def A_L0(i):
  return A(i, c=2) * Fraction(2, 3)**i

@lru_cache(maxsize=None)
def A_L1(i, n=3):
  if i <= 3:
    return 0
  p = 0
  for j in range(i):
    l = j
    r = i - j - 1
    l_not_r = A_L0(l)*(1 - A(r, c=2))*Fraction(2, 3)**r
    r_not_l = A_L0(r)*(1 - A(l, c=2))*Fraction(2, 3)**l
    l_and_r = A(l, c=2)*A(r, c=2)*Fraction(2, 3)**(i-1)
    subp = (l_not_r + r_not_l + l_and_r)*Fraction(1, 3)
    #print(j, l, r, lp*Fraction(1, 3), rp*Fraction(1, 3))
    p += subp
  return p

@lru_cache(maxsize=None)
def B(i):
  """Probability of being late at least twice."""
  if i < 2:
    return 0
  return 1 - (Fraction(2, 3)**i + i*Fraction(2**(i-1), 3**i))

def num_prize_strings(i, c=3):
  p1 = A_L0(i)
  p2 = A_L1(i)
  p3 = B(i)
  #print(f'A_L0: {p1}, A_L1: {p2}, B: {p3}')
  return c**i * (1 - p1 - p2 - p3) 

In [ ]:
A(1, c=2)

0

In [ ]:
A_L1(7)

Fraction(37, 729)

In [ ]:
%%time
for i in range(3, 31):
  print(i, A(i)*3**i, B(i)*3**i, num_prize_strings(i))

3 1 7 19
4 5 33 43
5 21 131 94
6 81 473 200
7 295 1611 418
8 1037 5281 861
9 3555 16867 1753
10 11961 52905 3536
11 39667 163835 7077
12 130049 502769 14071
13 422403 1532883 27820
14 1361385 4651897 54736
15 4359115 14070379 107236
16 13880129 42456897 209305
17 43984227 127894979 407167
18 138795849 384799049 789720
19 436367131 1156756443 1527607
20 1367434577 3475250065 2947811
21 4272615603 10436235955 5675882
22 13315096089 31330727961 10906776
23 41397076939 94038321227 20920006
24 128429930465 282211432673 40058421
25 397665266595 846835624611 76585973
26 1229127726825 2540926304233 146210464
27 3792875384251 7623651327931 278757449
28 11686625364785 22872765923121 530803311
29 35959122780051 68622055865747 1009569304
30 110502844543161 205873952225465 1918080160
CPU times: user 52.3 ms, sys: 171 µs, total: 52.5 ms
Wall time: 53.1 ms


In [ ]:
found = 0
total = 0
for c in sorted(itertools.product('ALO', repeat=7)):
  s = ''.join(c)
  if 'AAA' in s and s.count('L') == 1:
    found += 1
    #print(s)
  total += 1
print(Fraction(found, total))

37/729


In [ ]:
37*3

111

In [ ]:
def primes(n):
    """
    credit goes to someone else for this; it ended up being faster than my
    function above
    """
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = list(range(3, n+1, 2))
    mroot = n ** 0.5
    half = (n+1)//2-1
    i = 0
    m = 3
    while m <= mroot:
        if s[i]:
            j = (m*m-3)//2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i+1
        m = 2*i+3
    return [2]+[x for x in s if x]

In [ ]:
def iter_len(iter):
  count = 0
  for i in iter:
    count += 1
  return count

In [ ]:
primes = primes(100)

In [ ]:
def choose(n, k):
    return factorial(n) // (factorial(k) * factorial(n - k))

_factorials = [1, 1, 2]

def factorial(n):
    if n <= len(_factorials) - 1:
        return _factorials[n]
    else:
        start = len(_factorials)
        for idx in range(start, n + 1):
            _factorials.append(idx * _factorials[idx - 1])
        return _factorials[-1]

In [ ]:
for i in range(100):
  factorial(i)

In [ ]:
choose(4, 2)

6

In [ ]:
@lru_cache(maxsize=None)
def strings(consecutive, count, days):
  print(f'strings(consecutive={consecutive}, count={count}, days={days})')
  total = double_counted = 0
  free = days - consecutive * count
  if free < 0: return total
  
  if consecutive == 0:
    total = 1
  elif count == 1:
    total = days - consecutive + 1 
  else:
    total = choose(math.ceil(days / consecutive), count)
    for m in range(2, count + 1):
        if count % m != 0: continue
        double_counted += strings(consecutive * m, count // m, days)
        print('  -', consecutive*m, count//m, strings(consecutive * m, count // m, days))

  # late once + late zero times
  expanded = total * (free + 1)
  total_strings = expanded - double_counted 
  print(f'  -> total: {total}')
  print(f'  -> free: {free}')
  print(f'  -> expanded: {expanded}')
  print(f'  -> double_counted: {double_counted}')
  print(f'  -> total_strings: {total_strings}')
  print(f'###')
  return total_strings

def prize_strings(days):
  total = strings(0, 0, days)
  for consecutive in range(1, 3):
    for count in range(1, round(days / consecutive) + 1):
      total += strings(consecutive, count, days)
      double_counted = 0
      #print(consecutive, count, strings(consecutive, count, days))
      # for m in range(2, math.ceil(count / consecutive) + 1):
      #   if count % m != 0: continue
      #   double_counted += strings(consecutive * m, count // m, days)
      #   print('  -', consecutive*m, count//m, strings(consecutive * m, count // m, days))
      total -= double_counted
  return total

def prize_strings_naive(days):
  total = 0
  solutions = set()
  for c in itertools.product('ALO', repeat=days):
    s = ''.join(c)
    if 'AAA' not in s and s.count('L') <= 1:
      total += 1
      solutions.add(s)
  return total, solutions

In [ ]:
strings(3, 1, 5)

9

In [ ]:
for days in range(4, 7):
  print(f'### Days: {days}')
  total = prize_strings(days)
  print(f'Prize strings for days = {days}: {total}')
  print()

### Days: 4
strings(consecutive=0, count=0, days=4)
  -> total: 1
  -> free: 4
  -> expanded: 5
  -> double_counted: 0
  -> total_strings: 5
###
strings(consecutive=1, count=1, days=4)
  -> total: 4
  -> free: 3
  -> expanded: 16
  -> double_counted: 0
  -> total_strings: 16
###
strings(consecutive=1, count=2, days=4)
strings(consecutive=2, count=1, days=4)
  -> total: 3
  -> free: 2
  -> expanded: 9
  -> double_counted: 0
  -> total_strings: 9
###
  - 2 1 9
  -> total: 6
  -> free: 2
  -> expanded: 18
  -> double_counted: 9
  -> total_strings: 9
###
strings(consecutive=1, count=3, days=4)
strings(consecutive=3, count=1, days=4)
  -> total: 2
  -> free: 1
  -> expanded: 4
  -> double_counted: 0
  -> total_strings: 4
###
  - 3 1 4
  -> total: 4
  -> free: 1
  -> expanded: 8
  -> double_counted: 4
  -> total_strings: 4
###
strings(consecutive=1, count=4, days=4)
strings(consecutive=2, count=2, days=4)
strings(consecutive=4, count=1, days=4)
  -> total: 1
  -> free: 0
  -> expanded: 1
  -

In [ ]:
for days in range(1, 10):
  total, solutions = prize_strings_naive(days)
  print(f'Prize strings for days = {days}: {total}')
  # by_a_count = defaultdict(int)
  # kept = set()
  # for s in solutions:
  #   by_a_count[s.count('A')] += 1
  #   if s.count('A') == 4:
  #     kept.add(s)
  # print(len(kept))
  # print('\n'.join(sorted(kept)))
  # print(by_a_count)
  print()

Prize strings for days = 1: 3

Prize strings for days = 2: 8

Prize strings for days = 3: 19

Prize strings for days = 4: 43

Prize strings for days = 5: 94

Prize strings for days = 6: 200

Prize strings for days = 7: 418

Prize strings for days = 8: 861

Prize strings for days = 9: 1753

